In [ ]:
import pandas as pd
import numpy as np
import joblib
import warnings
from IPython.display import display

warnings.filterwarnings('ignore')

print("=======================================================")
print("   🤖 QUANT ENGINE v15 (SMART HYBRID SELECTION)        ")
print("=======================================================")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
BANKROLL = 30.00        
MIN_EDGE = 0.02           
MAX_ODDS_CAP = 6.00       
MAX_STAKE_PCT = 0.10      
KELLY_FRACTION = 0.25     

# --- FIXTURES ---
fixtures = [
    {"Home": "Manchester United", "Away": "Newcastle United", "Odds_1": 2.53, "Odds_X": 3.61, "Odds_2": 2.68},
    {"Home": "Nottingham Forest", "Away": "Manchester City", "Odds_1": 5.29, "Odds_X": 4.45, "Odds_2": 1.58},
    {"Home": "Arsenal",           "Away": "Brighton & Hove Albion", "Odds_1": 1.40, "Odds_X": 4.92, "Odds_2": 7.85},
    {"Home": "Brentford",         "Away": "Bournemouth",      "Odds_1": 2.29, "Odds_X": 3.60, "Odds_2": 3.03},
    {"Home": "Burnley",           "Away": "Everton",          "Odds_1": 4.01, "Odds_X": 3.35, "Odds_2": 2.01},
    {"Home": "Liverpool",         "Away": "Wolverhampton Wanderers", "Odds_1": 1.24, "Odds_X": 6.59, "Odds_2": 11.28},
    {"Home": "West Ham United",   "Away": "Fulham",           "Odds_1": 2.68, "Odds_X": 3.41, "Odds_2": 2.63},
    {"Home": "Chelsea",           "Away": "Aston Villa",      "Odds_1": 1.85, "Odds_X": 3.88, "Odds_2": 4.03},
    {"Home": "Sunderland",        "Away": "Leeds United",     "Odds_1": 2.58, "Odds_X": 3.23, "Odds_2": 2.86},
    {"Home": "Crystal Palace",    "Away": "Tottenham Hotspur", "Odds_1": 2.29, "Odds_X": 3.30, "Odds_2": 3.25},
]

# ==============================================================================
# 2. LOAD MODEL
# ==============================================================================
try:
    artifacts = joblib.load('football_model_final.pkl')
    model = artifacts['model']
    features = artifacts['features']
    current_elos = artifacts['elo_dict']
    df_recent = artifacts['df_recent']
    print(f"✅ Model Loaded.")
except:
    print("❌ Error loading pickle.")
    exit()

def get_stats(team):
    rows = df_recent[(df_recent['home_team_name'] == team) | (df_recent['away_team_name'] == team)]
    if len(rows) == 0: return None
    last = rows.sort_values('date').iloc[-1]
    prefix = 'home_' if last['home_team_name'] == team else 'away_'
    stats = {}
    roll_feats = ['team_xg', 'team_possession', 'shots_onTarget', 'corners', 'team_points', 'fouls', 'team_score']
    for f in roll_feats:
        col = f"{prefix}roll_{f}"
        stats[f] = last[col] if col in last else 0
    return stats

# ==============================================================================
# 3. ANALYSIS LOOP
# ==============================================================================
opportunities = []
ELITE_TEAMS = ["Manchester City", "Liverpool", "Arsenal"]

print(f"\n🔍 ANALYZING {len(fixtures)} MATCHES...")

for f in fixtures:
    h_team, a_team = f['Home'], f['Away']
    h_stats, a_stats = get_stats(h_team), get_stats(a_team)
    h_elo, a_elo = current_elos.get(h_team, 1500), current_elos.get(a_team, 1500)
    
    if not h_stats or not a_stats: continue

    input_data = {
        'diff_elo': (h_elo + 60) - a_elo, 
        'home_elo': h_elo, 'away_elo': a_elo, 'diff_rest': 0, 
        'home_roll_team_xg': h_stats['team_xg'], 'away_roll_team_xg': a_stats['team_xg'],
        'home_roll_team_possession': h_stats['team_possession'], 'away_roll_team_possession': a_stats['team_possession'],
        'home_roll_shots_onTarget': h_stats['shots_onTarget'], 'away_roll_shots_onTarget': a_stats['shots_onTarget'],
        'home_roll_corners': h_stats['corners'], 'away_roll_corners': a_stats['corners'],
        'home_roll_fouls': h_stats['fouls'], 'away_roll_fouls': a_stats['fouls']
    }
    
    roll_feats = ['team_xg', 'team_possession', 'shots_onTarget', 'corners', 'team_points', 'fouls']
    for feat in roll_feats:
        input_data[f"diff_{feat}"] = h_stats[feat] - a_stats[feat]

    input_df = pd.DataFrame([input_data]).reindex(columns=features, fill_value=0)
    probs = model.predict_proba(input_df)[0]
    p_away, p_draw, p_home = probs[0], probs[1], probs[2]

    # --- 1. PROBABILITY CALCULATIONS ---
    # Raw Probs (For Winners)
    raw_p_home, raw_p_away = p_home, p_away
    

    # --- 2. GOLIATH FILTER ---
    block_underdog = False
    if (a_team in ELITE_TEAMS and h_elo < a_elo - 100) or (h_team in ELITE_TEAMS and a_elo < h_elo - 100):
        block_underdog = True

    print(f"{h_team[:10]} vs {a_team[:10]}: H {p_home:.2f} | D {p_draw:.2f} | A {p_away:.2f}" )

    # --- 3. VALUE FINDER ---
    bets = []

    if abs(p_home - p_away)<= 0.05:
            boosted_p_draw =  0.40  
            p_draw = boosted_p_draw
            p_home = 0.30 + (0.50-p_home) if p_home>p_away else 0.30 - (0.50-p_home)
            p_away = 0.30 + (0.50-p_away) if p_home<p_away else 0.30 - (0.50-p_away)
            print(f"Boosted Draw fixture {h_team} vs {a_team}: | H: {p_home:.2f} | D: {p_draw:.2f} | A: {p_away:.2f}")
    
    # HOME (Use Raw Prob)
    if f['Odds_1'] < MAX_ODDS_CAP and not (block_underdog and p_home < 0.40):
        edge = (p_home* f['Odds_1']) - 1
        # print("HOME Edge:",edge)
        if edge > MIN_EDGE and p_home > 0.20:
            bets.append({'Type': 'HOME', 'Team': h_team, 'Odds': f['Odds_1'], 'Prob': p_home, 'Edge': edge})

    if f['Odds_X'] < MAX_ODDS_CAP:
        edge = (p_draw * f['Odds_X']) - 1
        # print("DRAW Edge:",edge)
        if edge > MIN_EDGE and p_draw > 0.23:
            bets.append({'Type': 'DRAW', 'Team': 'Draw', 'Odds': f['Odds_X'], 'Prob': p_draw, 'Edge': edge})

    # AWAY (Use Raw Prob)
    if f['Odds_2'] < MAX_ODDS_CAP and not (block_underdog and p_away < 0.40):
        edge = (raw_p_away * f['Odds_2']) - 1
        # print("AWAY Edge:",edge)
        if edge > MIN_EDGE and raw_p_away > 0.20:
            bets.append({'Type': 'AWAY', 'Team': a_team, 'Odds': f['Odds_2'], 'Prob': raw_p_away, 'Edge': edge})

    # --- 4. SELECTION LOGIC ---
    if bets:
        # Sort by Edge initially
        best = sorted(bets, key=lambda x: x['Edge'], reverse=True)[0]
        
        best['Match'] = f"{h_team} vs {a_team}"
        opportunities.append(best)

# ==============================================================================
# 4. PORTFOLIO
# ==============================================================================
if opportunities:
    res = pd.DataFrame(opportunities)
    
    def calculate_stake(row):
        b = row['Odds'] - 1
        p = row['Prob']
        q = 1 - p
        f = ((b * p) - q) / b
        return max(0, min(f * KELLY_FRACTION, MAX_STAKE_PCT))

    res['Stake_Pct'] = res.apply(calculate_stake, axis=1)
    res['Stake'] = res['Stake_Pct'] * BANKROLL
    res['Profit'] = res['Stake'] * (res['Odds'] - 1)
    
    res = res[res['Stake'] > 0.01].sort_values('Stake', ascending=False)
    
    display_df = res[['Match', 'Type', 'Team', 'Odds', 'Prob', 'Edge', 'Stake', 'Profit']].copy()
    display_df.columns = ['Match', 'Bet Type', 'Selection', 'Odds', 'Win %', 'Edge', 'Stake ($)', 'Profit ($)']
    
    styled = (display_df.style
        .format({'Odds': '{:.2f}', 'Win %': '{:.1%}', 'Edge': '{:.1%}', 'Stake ($)': '${:.2f}', 'Profit ($)': '${:.2f}'})
        .background_gradient(subset=['Win %'], cmap='Greens', vmin=0.2, vmax=0.8)
        .background_gradient(subset=['Edge'], cmap='Blues', vmin=0.0, vmax=0.5)
        .bar(subset=['Stake ($)'], color='#ff6b6b', vmin=0)
        .bar(subset=['Profit ($)'], color='#51cf66', vmin=0)
        .set_caption(f"🚀 FINAL QUANT SLIP v15 (Value + Safety) | Bankroll: ${BANKROLL}")
    )
    display(styled)
    
    print(f"\n💰 Total Risk: ${res['Stake'].sum():.2f}")
else:
    print("\n📉 No Value Bets Found.")

   🤖 QUANT ENGINE v15 (SMART HYBRID SELECTION)        
✅ Model Loaded.

🔍 ANALYZING 10 MATCHES...
Manchester vs Newcastle : H 0.46 | D 0.23 | A 0.31
HOME Edge: 0.16926438181935533
DRAW Edge: -0.1703470065022944
AWAY Edge: -0.1703470065022944
Nottingham vs Manchester: H 0.33 | D 0.06 | A 0.61
DRAW Edge: -0.7311706976737742
AWAY Edge: -0.7311706976737742
Arsenal vs Brighton &: H 0.64 | D 0.22 | A 0.15
HOME Edge: -0.1098817545663815
DRAW Edge: 0.07177278625120165
Brentford vs Bournemout: H 0.41 | D 0.26 | A 0.33
HOME Edge: -0.062197172774731024
DRAW Edge: -0.05398508483591036
AWAY Edge: -0.05398508483591036
Burnley vs Everton: H 0.45 | D 0.07 | A 0.48
Boosted Draw fixture Burnley vs Everton: | H: 0.25 | D: 0.40 | A: 0.32
HOME Edge: -0.01634900462287181
DRAW Edge: 0.3400000000000001
AWAY Edge: 0.3400000000000001
Liverpool vs Wolverhamp: H 0.75 | D 0.06 | A 0.19
HOME Edge: -0.06389841872066271
West Ham U vs Fulham: H 0.40 | D 0.11 | A 0.49
HOME Edge: 0.07872292003623294
DRAW Edge: -0.620813

,Match,Bet Type,Selection,Odds,Win %,Edge,Stake ($),Profit ($)
5,Crystal Palace vs Tottenham Hotspur,HOME,Crystal Palace,2.29,57.7%,32.2%,$1.87,$2.42
3,Chelsea vs Aston Villa,AWAY,Aston Villa,4.03,40.7%,64.1%,$1.59,$4.81
4,Sunderland vs Leeds United,HOME,Sunderland,2.58,50.6%,30.5%,$1.45,$2.29
2,West Ham United vs Fulham,AWAY,Fulham,2.63,48.6%,27.9%,$1.28,$2.09
1,Burnley vs Everton,DRAW,Draw,3.35,40.0%,34.0%,$1.09,$2.55
0,Manchester United vs Newcastle United,HOME,Manchester United,2.53,46.2%,16.9%,$0.83,$1.27



💰 Total Risk: $8.11
